In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, LSTM, Dense, Dropout, BatchNormalization, MaxPooling1D
from tensorflow.keras.datasets import imdb

# Load IMDb dataset
vocab_size = 10000  # Keep top 10,000 words
max_len = 200  # Maximum sequence length

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences
x_train = pad_sequences(x_train, maxlen=max_len, padding='post')
x_test = pad_sequences(x_test, maxlen=max_len, padding='post')

# Build the DSCNN Model
def build_ds_cnn_model():
    inputs = Input(shape=(max_len,))

    # Embedding Layer
    x = Embedding(input_dim=vocab_size, output_dim=128)(inputs)

    # CNN Layer for Feature Extraction
    x = Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)  # Reduce sequence length

    # LSTM Layer for Long-Term Dependencies (Passes 3D input correctly)
    x = LSTM(64, return_sequences=False)(x)

    # Fully Connected Layers
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)  # Binary classification

    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train the Model
model = build_ds_cnn_model()
model.summary()

history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Evaluate Model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"\nTest Accuracy: {test_acc:.4f}")


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 200, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 200, 128)            │          82,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 200, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 100, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,416,193 (5.40 MB)

 Trainable params: 1,415,937 (5.40 MB)

 Non-trainable params: 256 (1.00 KB)

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 153s 382ms/step - accuracy: 0.6138 - loss: 0.6186 - val_accuracy: 0.7894 - val_loss: 0.4953
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 200s 378ms/step - accuracy: 0.9029 - loss: 0.2572 - val_accuracy: 0.8850 - val_loss: 0.2855
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 200s 373ms/step - accuracy: 0.9494 - loss: 0.1442 - val_accuracy: 0.8773 - val_loss: 0.3374
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 202s 373ms/step - accuracy: 0.9748 - loss: 0.0821 - val_accuracy: 0.8736 - val_loss: 0.3739
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 202s 372ms/step - accuracy: 0.9879 - loss: 0.0425 - val_accuracy: 0.8670 - val_loss: 0.4533
782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 39ms/step - accuracy: 0.8675 - loss: 0.4485

Test Accuracy: 0.8670
